In [6]:
# Ch14-3-containers

In [7]:
# Build and simple Docker container and push it to AWS ECR 

In [34]:
# Import Libraries
import os
import boto3
import subprocess
from botocore.exceptions import ClientError
import logging

In [25]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [26]:
# AWS Settings
AWS_REGION = 'us-east-1'  # Change to your preferred region
REPOSITORY_NAME = 'my-simple-app'  # Name for your ECR repository
IMAGE_TAG = 'latest'

# Create a unique ID for the build
#BUILD_ID = str(uuid.uuid4())[:8]

In [27]:
def create_dockerfile():
    """Create a simple Dockerfile for a Python application"""
    logger.info("Creating Dockerfile and application files...")
    
    # Create a directory for our app
    os.makedirs('app', exist_ok=True)
    
    # Create a simple Flask application
    with open('app/app.py', 'w') as f:
        f.write('''
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello():
    return "Hello from the Docker container!"

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)
''')
    
    # Create requirements.txt
    with open('app/requirements.txt', 'w') as f:
        f.write('flask==2.2.3\n')
    
    # Create Dockerfile
    with open('Dockerfile', 'w') as f:
        f.write('''
FROM python:3.9-slim

WORKDIR /app

COPY app/requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY app/ .

EXPOSE 5000

CMD ["python", "app.py"]
''')
    
    logger.info("Dockerfile and application files created successfully")

In [28]:
def build_docker_image():
    """Build the Docker image locally"""
    image_name = f"{REPOSITORY_NAME}:{IMAGE_TAG}"
    logger.info(f"Building Docker image: {image_name}")
    
    try:
        subprocess.run(
            ["docker", "build", "-t", image_name, "."],
            check=True,
            stdout=subprocess.PIPE,
            text=True
        )
        logger.info("Docker image built successfully")
        return image_name
    except subprocess.CalledProcessError as e:
        logger.error(f"Failed to build Docker image: {e}")
        raise

In [29]:
def create_ecr_repository(ecr_client):
    """Create an ECR repository if it doesn't exist"""
    try:
        logger.info(f"Checking if repository {REPOSITORY_NAME} exists...")
        ecr_client.describe_repositories(repositoryNames=[REPOSITORY_NAME])
        logger.info(f"Repository {REPOSITORY_NAME} already exists")
    except ecr_client.exceptions.RepositoryNotFoundException:
        logger.info(f"Creating repository {REPOSITORY_NAME}...")
        response = ecr_client.create_repository(
            repositoryName=REPOSITORY_NAME,
            imageScanningConfiguration={'scanOnPush': True},
            encryptionConfiguration={'encryptionType': 'AES256'}
        )
        logger.info(f"Repository created: {response['repository']['repositoryUri']}")

In [30]:
def get_ecr_login_command(ecr_client):
    """Get the ECR login command"""
    response = ecr_client.get_authorization_token()
    
    auth_data = response['authorizationData'][0]
    token = auth_data['authorizationToken']
    endpoint = auth_data['proxyEndpoint']
    
    return token, endpoint

In [31]:
def tag_and_push_image(local_image_name, repository_uri):
    """Tag and push the Docker image to ECR"""
    logger.info(f"Tagging image as {repository_uri}")
    
    # Tag the image
    try:
        subprocess.run(
            ["docker", "tag", local_image_name, repository_uri],
            check=True,
            stdout=subprocess.PIPE,
            text=True
        )
        logger.info("Image tagged successfully")
    except subprocess.CalledProcessError as e:
        logger.error(f"Failed to tag image: {e}")
        raise
    
    # Push the image
    try:
        logger.info(f"Pushing image to {repository_uri}")
        subprocess.run(
            ["docker", "push", repository_uri],
            check=True,
            stdout=subprocess.PIPE,
            text=True
        )
        logger.info("Image pushed successfully")
    except subprocess.CalledProcessError as e:
        logger.error(f"Failed to push image: {e}")
        raise


In [32]:
def login_to_ecr(region, endpoint):
    try:
        print(f"Logging into ECR at {endpoint}...")
        subprocess.run(
            f"aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {endpoint.replace('https://', '')}",
            shell=True,
            check=True
        )
        print("Logged in to ECR successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Failed to log in to ECR: {e}")
        raise

In [33]:
def main():
    """Main function to build and push a Docker image to ECR"""
    try:
        # Create sample application and Dockerfile
        create_dockerfile()
        
        # Build the Docker image locally
        local_image_name = build_docker_image()
        
        # Initialize boto3 ECR client
        logger.info(f"Connecting to AWS ECR in region {AWS_REGION}...")
        ecr_client = boto3.client('ecr', region_name=AWS_REGION)
        
        # Create the ECR repository if it doesn't exist
        create_ecr_repository(ecr_client)
        
        # Get the repository URI
        response = ecr_client.describe_repositories(repositoryNames=[REPOSITORY_NAME])
        repository_uri = response['repositories'][0]['repositoryUri']
        repository_uri_with_tag = f"{repository_uri}:{IMAGE_TAG}"
        
        # Get ECR authorization token
        token, endpoint = get_ecr_login_command(ecr_client)

        # Use the AWS CLI to log in to ECR
        logger.info("Logging in to ECR...")
        login_to_ecr(AWS_REGION, endpoint.replace("https://", ""))
                
        # Tag and push the image
        tag_and_push_image(local_image_name, repository_uri_with_tag)
        
        logger.info(f"Successfully built and pushed Docker image to: {repository_uri_with_tag}")
        logger.info("You can pull this image using:")
        logger.info(f"docker pull {repository_uri_with_tag}")
        
        return repository_uri_with_tag
    
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        raise

if __name__ == "__main__":
    main()

2025-05-17 15:09:33,419 - INFO - Creating Dockerfile and application files...
2025-05-17 15:09:33,423 - INFO - Dockerfile and application files created successfully
2025-05-17 15:09:33,424 - INFO - Building Docker image: my-simple-app:latest
#0 building with "desktop-linux" instance using docker driver

#1 [internal] load .dockerignore
#1 transferring context: 2B done
#1 DONE 0.0s

#2 [internal] load build definition from Dockerfile
#2 transferring dockerfile: 207B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.9-slim
#3 DONE 0.8s

#4 [1/5] FROM docker.io/library/python:3.9-slim@sha256:bef8d69306a7905f55cd523f5604de1dde45bbf745ba896dbb89f6d15c727170
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 318B done
#5 DONE 0.0s

#6 [2/5] WORKDIR /app
#6 CACHED

#7 [3/5] COPY app/requirements.txt .
#7 CACHED

#8 [4/5] RUN pip install --no-cache-dir -r requirements.txt
#8 CACHED

#9 [5/5] COPY app/ .
#9 CACHED

#10 exporting to image
#10 expo

Logging into ECR at 495163878159.dkr.ecr.us-east-1.amazonaws.com...


2025-05-17 15:09:37,611 - INFO - Tagging image as 495163878159.dkr.ecr.us-east-1.amazonaws.com/my-simple-app:latest
2025-05-17 15:09:37,667 - INFO - Image tagged successfully
2025-05-17 15:09:37,668 - INFO - Pushing image to 495163878159.dkr.ecr.us-east-1.amazonaws.com/my-simple-app:latest


Login Succeeded
Logged in to ECR successfully.


2025-05-17 15:09:47,812 - INFO - Image pushed successfully
2025-05-17 15:09:47,813 - INFO - Successfully built and pushed Docker image to: 495163878159.dkr.ecr.us-east-1.amazonaws.com/my-simple-app:latest
2025-05-17 15:09:47,814 - INFO - You can pull this image using:
2025-05-17 15:09:47,814 - INFO - docker pull 495163878159.dkr.ecr.us-east-1.amazonaws.com/my-simple-app:latest


In [18]:
## Now remove the container image from ECR ##

In [35]:
def delete_ecr_image(repo_name, image_tag=None, image_digest=None, region="us-east-1"):
    """
    Deletes an image from an ECR repository using either a tag or a digest.
    """
    ecr = boto3.client('ecr', region_name=region)

    if not image_tag and not image_digest:
        print("You must provide either an image_tag or image_digest.")
        return

    image_id = {}
    if image_tag:
        image_id['imageTag'] = image_tag
    if image_digest:
        image_id['imageDigest'] = image_digest

    try:
        response = ecr.batch_delete_image(
            repositoryName=repo_name,
            imageIds=[image_id]
        )
        failures = response.get("failures", [])
        if failures:
            print("Failures:", failures)
        else:
            print(f"Successfully deleted image from '{repo_name}': {image_id}")
    except ClientError as e:
        print(f"Failed to delete image: {e}")
        raise


In [36]:
# ---- Usage Example ----
repo_name = "my-simple-app"         # Replace with your ECR repository name
image_tag = "latest"                      # Replace with the tag you want to delete
delete_ecr_image(repo_name, image_tag=image_tag)


Successfully deleted image from 'my-simple-app': {'imageTag': 'latest'}


In [37]:
def delete_ecr_repository(repo_name, region="us-east-1"):
    """Delete the ECR repository itself after it's empty."""
    ecr = boto3.client('ecr', region_name=region)
    try:
        ecr.delete_repository(repositoryName=repo_name, force=False)
        print(f"Repository '{repo_name}' deleted successfully.")
    except ClientError as e:
        print(f"Failed to delete repository: {e}")
        raise

In [38]:
delete_ecr_repository("my-simple-app", "us-east-1")

Repository 'my-simple-app' deleted successfully.


In [ ]:
## End of Notebook ##